In [54]:
import pandas as pd
import pyarc
from pyids import IDS
from pyids.ids_classifier import mine_CARs

In [55]:
from pyarc.qcba.transformation import QCBATransformation

from pyarc import CBA, TransactionDB
from pyarc.qcba.data_structures import QuantitativeDataFrame
import pandas as pd

from pyarc.qcba.data_structures import (
    IntervalReader,
    Interval,
    QuantitativeDataFrame,
    QuantitativeCAR
)

from pyarc.qcba.classifier import QuantitativeClassifier
from sklearn.metrics import accuracy_score

interval_reader = IntervalReader()

interval_reader.closed_bracket = "", "NULL"
interval_reader.open_bracket = "NULL", ""
interval_reader.infinity_symbol = "inf", "inf"
interval_reader.members_separator = "_to_"

interval_reader.compile_reader()

QuantitativeCAR.interval_reader = interval_reader

In [56]:
data_train_disc = pd.read_csv("c:/code/python/machine_learning/assoc_rules/train/iris0.csv")
data_train_undisc = pd.read_csv("c:/code/python/machine_learning/assoc_rules/folds_undiscr/train/iris0.csv")
data_test = pd.read_csv("c:/code/python/machine_learning/assoc_rules/test/iris0.csv")

In [57]:
quant_dataframe_train_disc = QuantitativeDataFrame(data_train_disc)
quant_dataframe_train_undisc = QuantitativeDataFrame(data_train_undisc)

In [72]:
cars = mine_CARs(data_train_disc, 50, sample=True)

Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=3, MAX_RULE_LEN=5
Rule count: 96, Iteration: 1
Increasing maxlen 4
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=4, MAX_RULE_LEN=5
Rule count: 166, Iteration: 2
Increasing maxlen 5
Running apriori with setting: confidence=0.5, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 3
Decreasing confidence to 0.45
Running apriori with setting: confidence=0.45, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 4
Decreasing confidence to 0.4
Running apriori with setting: confidence=0.4, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 5
Decreasing confidence to 0.35000000000000003
Running apriori with setting: confidence=0.35000000000000003, support=0.0, minlen=2, maxlen=5, MAX_RULE_LEN=5
Rule count: 190, Iteration: 6
Decreasing confidence to 0.30000000000000004
Running apriori with setting: confidence=0.30000000

In [73]:
ids = IDS()
ids.fit(class_association_rules=cars, quant_dataframe=quant_dataframe_train_disc, debug=False)

cover cache prepared
overlap cache prepared


In [74]:
rules_to_optimize = ids.clf.rules

In [75]:
quant_rules = [ QuantitativeCAR(r.car) for r in rules_to_optimize ]

In [76]:
qcba_transformation = QCBATransformation(quant_dataframe_train_undisc)
transformed_rules = qcba_transformation.transform(quant_rules)

applying all transformations
[                                                  ]
[###                                               ]
[#######                                           ]
[###########                                       ]
[###############                                   ]
[###################                               ]
[#######################                           ]
[##########################                        ]
[##############################                    ]
[##################################                ]
[######################################            ]
[##########################################        ]
[##############################################    ]


In [77]:
rules, default_class = transformed_rules


qclf = QuantitativeClassifier(rules, default_class)
pred = qclf.predict(quant_dataframe_train_undisc)


actual = quant_dataframe_train_undisc.dataframe["class"].values

In [78]:
print("Přednost IDS:", ids.score(quant_dataframe_train_disc))
print("Přednost IDS-QCBA:", accuracy_score(actual, pred))

Přednost IDS: 0.6666666666666666
Přednost IDS-QCBA: 0.9703703703703703


In [80]:
print("Délka IDS solution setu:", len(rules_to_optimize))
print("Délka IDS-QCBA solution setu:", len(rules))

Délka IDS solution setu: 13
Délka IDS-QCBA solution setu: 8
